In [1]:
# Importing modules...
import numpy as np

import sys
sys.path.append('..') # Go back to base directory

from modules.graph import *
from modules.viewer3d import *
from modules.dc_motor import *
from modules.pid_controller import *

In [2]:
# Instanciating and simulating the model
DCM = DCMotor(
    # Electrical parameters
    R_a=3.0, # In ohms
    L_a=6.0e-3, # In henrys
    K_t=50.0e-3, # In N*m/A

    # Mechanical Parameters
    J=100e-6, # In Kg*m^2
    b=105e-6, # In Kg*m^2/s

    dt=1e-4, # Differential time step

    # Initial angular velocity
    w=0.0 # In rad/s
)

PID = PID(
    K_p=5.0,
    K_i=7.0,
    K_d=0.01,
    dt=1e-4
)

# Simulation time parameters
start_time = 0.0 # In Seconds
stop_time = 2.0  # In Seconds

time_vector = np.linspace(
    start_time, 
    stop_time, 
    int((stop_time - start_time) / DCM.dt)
)

# Generate PWM signal for motor's input
def pwm_signal(
    V_cc, 
    duty_cycle, 
    frequency, 
    time_vector
):
    return V_cc * (time_vector % (1 / frequency) < (1 / frequency) * duty_cycle / 100)

# Generate step input
def step_signal(
    value,
    delay
):
    return value * np.heaviside(time_vector - delay, 1)

# PWM signal parameters 
V_cc = 20 # In volts
duty_cycle = 50 # In %
frequency = 490 # In Hz

v_signal = pwm_signal(V_cc, duty_cycle, frequency, time_vector)

# Torque signal parameters
T_load = 0.05 # In N*m
T_signal = step_signal(T_load, 1) # Starts at 1 second

# Reference angular velocity signal
r_signal = np.full(time_vector.shape, 250)

# Generate solution
output = []
for v, T, r, t in zip(v_signal, T_signal, r_signal, time_vector):
    w_r = DCM.get_output().ravel()[1]
    e = r - w_r # Error signal

    v = PID.get_output(e)

    DCM.electromechanical_model(v, T)

    output.append(DCM.get_output())

output = np.hstack(output)

In [3]:
# Plot input variables
graph = Graph(
    title="Input in Time Domain", 
    axis_title=("Time (s)", "Armature Voltage (V)")
)

timespan = round(5/(frequency * DCM.dt)) # Numbers of samples for a timespan of 5 cycles

graph.add_trace(
    points=np.vstack((time_vector, v_signal))[:, :timespan], 
    name="v (V)"
)

graph.figure.show(renderer='notebook_connected')

In [4]:
# Plot input variables
graph = Graph(
    title="Input in Time Domain", 
    axis_title=("Time (s)", "Load Torque (N*m)")
)

graph.add_trace(
    points=np.vstack((time_vector, T_signal)), 
    name="T (N*m)"
)

graph.figure.show(renderer='notebook_connected')

In [5]:
# Plot output variables
graph = Graph(
    title="Output in Time Domain", 
    axis_title=("Time (s)", "Armature Current (A)")
)

graph.add_trace(
    points=np.vstack((time_vector, output[0])), 
    name="i_a (A)"
)

graph.figure.show(renderer='notebook_connected')

In [6]:
# Plot output variables
graph = Graph(
    title="Output in Time Domain", 
    axis_title=("Time (s)", "Rotor Angular Velocity (rad/s)")
)

graph.add_trace(
    points=np.vstack((time_vector, output[1])), 
    name="w (rad/s)"
)

graph.figure.show(renderer='notebook_connected')